In [7]:
#Import the Library
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

In [8]:
#Create a chrome driver
driver = webdriver.Chrome(executable_path=r'chromedriver')

#Getting the Lottery MegaMillion website
url = "https://www.megamillions.com/Winning-Numbers/Previous-Drawings.aspx?pageNumber=78&pageSize=12&startDate=&endDate="
driver.get(url)




## Getting all pages from Mega Million website

In [3]:
# Create an empty list for saving the pages
pages=[]

#Collecting the whole pages in the lottery website
for i in range(1,9):

    for i,page in enumerate(driver.find_element_by_class_name("pageLinks").find_elements_by_tag_name('a')):
        #collecting the pages href links for the each 10 pages
        pages.append(page.get_attribute('href'))
    #Add the next page link to the data
    pages.append(driver.find_element_by_class_name('nextLinks').find_element_by_tag_name('a').get_attribute('href'))
    driver.find_element_by_class_name('nextLinks').find_element_by_tag_name('a').click()
    time.sleep(2)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"tag name","selector":"a"}
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.42.591088 (7b2b2dca23cca0862f674758c9a3933e685c27d5),platform=Windows NT 10.0.17134 x86_64)


## Scraping the date, first_ball, second_ball, third_ball,fourth_ball, fifth_ball, yellow_ball

In [ ]:
#Create list for each paramter
date=[]
first_no=[]
second_no=[]
third_no=[]
fourth_no=[]
fifth_no=[]
yellow_no=[]

# Iterating for each winning number in each page
for page in pages:
    driver.get(page)
    for i in driver.find_elements_by_class_name("prevDrawItem"):
        #Adding the collecting dates to a date list
        date.append(i.find_element_by_class_name("drawItemDate").text)
        
        #Adding the first numbers to the first_no list
        first_no.append(int(i.find_element_by_css_selector('.ball.pastNum1').text))
        
        #Adding the second numbers to the second_no list
        second_no.append(int(i.find_element_by_css_selector('.ball.pastNum2').text))
        
        #Adding the third numbers to the third_no list
        third_no.append(int(i.find_element_by_css_selector('.ball.pastNum3').text))
        
        #Adding the fourth numbers to the fourth_no list
        fourth_no.append(int(i.find_element_by_css_selector('.ball.pastNum4').text))
        
        #Adding the fifth numbers to the fifth_no list
        fifth_no.append(int(i.find_element_by_css_selector('.ball.pastNum5').text))
        
        #Adding the yellow numbers to the yellow_no list
        yellow_no.append(int(i.find_element_by_css_selector('.ball.yellowBall.pastNumMB').text))
    
    #Waiting time for page uploading
    time.sleep(2)

    

## Create DataFrame for the collected data

In [ ]:
# Create data frame for the results with naming the colmuns

df = pd.DataFrame({'date':date,'first':first_no,'second':second_no,'third':third_no,'fourth':fourth_no,'fifth':fifth_no,'Yellow':yellow_no})

### Create csv file for the data frame and name as "Lottery_data"

In [ ]:
df.to_csv('Lottery_data.csv')

In [9]:
df = pd.read_csv('Lottery_data.csv')
df['result']=1
df.head()

,Unnamed: 0,Yellow,date,fifth,first,fourth,second,third,result
0,0,20,1/11/2019,69,4,62,5,31,1
1,1,5,1/8/2019,64,13,38,26,29,1
2,2,15,1/4/2019,60,21,54,29,35,1
3,3,14,1/1/2019,70,34,62,44,57,1
4,4,21,12/28/2018,38,9,37,10,25,1


## Create Random FALSE numbers

In [31]:
import random
first= []
second=[]
third=[]
fourth=[]
fifth=[]
sixth=[]
result=[]

for x in range(1000):
    first.append(random.randint(1,70))
    second.append(random.randint(1,70))
    third.append(random.randint(1,70))
    fourth.append(random.randint(1,70))
    fifth.append(random.randint(1,70))
    sixth.append(random.randint(1,25))
    result.append(0)

## Create dataframe for the losing numbers

In [32]:
df_loser= pd.DataFrame({'first':first,
                       'second':second,
                       'third':third,
                        'fourth':fourth,
                        'fifth':fifth,
                        'Yellow':sixth,
                        'result':result
                       }) 

### Concatinate the winning numbers with loosing ones


In [33]:
frames=[df,df_loser]
result = pd.concat(frames)

In [34]:
result.head()

,Unnamed: 0,Yellow,date,fifth,first,fourth,result,second,third
0,0.0,20,1/11/2019,69,4,62,1,5,31
1,1.0,5,1/8/2019,64,13,38,1,26,29
2,2.0,15,1/4/2019,60,21,54,1,29,35
3,3.0,14,1/1/2019,70,34,62,1,44,57
4,4.0,21,12/28/2018,38,9,37,1,10,25


## Bulid a machine learning model RANDOM FOREST 

In [35]:
#Import Random Forest Classifier from SKLEARN library
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, criterion='entropy',max_depth=10,random_state=0)

In [36]:
# Spilt and train the data
from sklearn.model_selection import train_test_split
X=result[['first','second','third','fourth','fifth','Yellow']]
Y= result['result']

x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2, random_state=42)

# Normalizing the data values
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [37]:
# Fit the data into the model
clf.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [38]:
#Predict the outcome
y_predict = clf.predict(x_test)

In [39]:
# measuring the results by condusion matrix
from sklearn.metrics import confusion_matrix
cm= confusion_matrix(y_test,y_predict)
print(cm)

[[178  18]
 [  3 207]]


In [40]:
#model score and accuracy
clf.score(x_test,y_test)

0.9482758620689655

## Predicting the winning numbers

In [44]:
import random
process = False
while process is False:
    first_nu =random.sample(range(1,70), 5)
    sixth_nu = random.randint(1,25)
    numbers = first_nu + [sixth_nu]
    re = clf.predict([numbers])
    if re == 1:
        
        print (f'Your winning numbers are: {numbers}')
        process = True
    
    

Your winning numbers are: [1, 11, 22, 3, 13, 18]


In [46]:
clf.predict([[1,4,6,11,14,30]])

array([1], dtype=int64)

In [59]:
first = input('Write your number')

Write your numberf
